In [1]:
versioninfo()

Julia Version 1.6.2
Commit 1b93d53fc4 (2021-07-14 15:36 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.7.0)
  CPU: Intel(R) Core(TM) i5-8279U CPU @ 2.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)


In [2]:
using Pkg
Pkg.activate("../..")
Pkg.status()

  Activating environment at `~/Dropbox/class/M1399.000200/2021/M1399_000200-2021fall/Project.toml`


      Status `~/Dropbox/class/M1399.000200/2021/M1399_000200-2021fall/Project.toml`
  [7d9fca2a] Arpack v0.5.3
  [6e4b80f9] BenchmarkTools v1.1.4
  [1e616198] COSMO v0.8.1
  [f65535da] Convex v0.14.13
  [a93c6f00] DataFrames v1.2.2
  [31a5f54b] Debugger v0.6.8
  [31c24e10] Distributions v0.24.18
  [e2685f51] ECOS v0.12.3
  [f6369f11] ForwardDiff v0.10.19
  [28b8d3ca] GR v0.58.1
  [c91e804a] Gadfly v1.3.3
  [bd48cda9] GraphRecipes v0.5.7
  [82e4d734] ImageIO v0.5.8
  [6218d12a] ImageMagick v1.2.1
  [916415d5] Images v0.24.1
  [b6b21f68] Ipopt v0.7.0
  [42fd0dbc] IterativeSolvers v0.9.1
  [4076af6c] JuMP v0.21.9
  [b51810bb] MatrixDepot v1.0.4
  [1ec41992] MosekTools v0.9.4
  [76087f3c] NLopt v0.6.3
  [47be7bcc] ORCA v0.5.0
  [a03496cd] PlotlyBase v0.4.3
  [f0f68f2c] PlotlyJS v0.15.0
  [91a5bcdd] Plots v1.21.2
  [438e738f] PyCall v1.92.3
  [d330b81b] PyPlot v2.9.0
  [dca85d43] QuartzImageIO v0.7.3
  [6f49c342] RCall v0.13.12
  [ce6b1742] RDatasets v0.7.5
  [c946c3f1] SCS v0.7.1
  [276daf

# Triangular systems

We consider computer algorithms for solving linear equations $\mathbf{A} \mathbf{x} = \mathbf{b}$, a ubiquitous task in statistics. 

Idea: turning original problem into an **easy** one, e.g., triangular system.  

(Note that the easiest one is diagonal system, which requires $n$ flops where simultaneous calculation of all $n$ elements is possible unlike triangular system.)

## Lower triangular system

To solve $\mathbf{A} \mathbf{x} = \mathbf{b}$, where $\mathbf{A} \in \mathbb{R}^{n \times n}$ is **lower triangular**

$$
\begin{pmatrix}
    a_{11} & 0 & \cdots & 0 \\
    a_{21} & a_{22} & \cdots & 0 \\
    \vdots & \vdots & \ddots & \vdots \\
    a_{n1} & a_{n2} & \cdots & a_{nn}
\end{pmatrix}
\begin{pmatrix}
x_1 \\ x_2 \\ \vdots \\ x_n
\end{pmatrix} = \begin{pmatrix}
b_1 \\ b_2 \\ \vdots \\ b_n
\end{pmatrix}.
$$

* **Forward substitution**: 
$$
\begin{align*}
    x_1 &= b_1 / a_{11} \\
    x_2 &= (b_2 - a_{21} x_1) / a_{22} \\
    x_3 &= (b_3 - a_{31} x_1 - a_{32} x_2) / a_{33} \\
    &\vdots \\
    x_n &= (b_n - a_{n1} x_1 - a_{n2} x_2 - \cdots - a_{n,n-1} x_{n-1}) / a_{nn}.
\end{align*}
$$

* $1 + 3 + 5 + \cdots + (2n-1) = n^2$ flops. $\rightarrow$ level 2 operation in BLAS

* $\mathbf{A}$ can be accessed by row ($ij$ loop) or column ($ji$ loop).

## Upper triangular system

To solve $\mathbf{A} \mathbf{x} = \mathbf{b}$, where $\mathbf{A} \in \mathbb{R}^{n \times n}$ is upper triangular  
$$
\begin{pmatrix}
    a_{11} & \cdots & a_{1,n-1} & a_{1n} \\
    \vdots & \ddots & \vdots & \vdots \\
    0 & \cdots & a_{n-1,n-1} & a_{n-1,n} \\
    0 & 0 & 0 & a_{nn}
\end{pmatrix}
\begin{pmatrix}
x_1 \\ \vdots \\ x_{n-1} \\ x_n
\end{pmatrix} = \begin{pmatrix}
b_1 \\ \vdots \\ b_{n-1} \\ b_n
\end{pmatrix}.
$$

* **Back substitution** (backsolve): 
$$
\begin{align*}
    x_n &= b_n / a_{nn} \\
    x_{n-1} &= (b_{n-1} - a_{n-1,n} x_n) / a_{n-1,n-1} \\
    x_{n-2} &= (b_{n-2} - a_{n-2,n-1} x_{n-1} - a_{n-2,n} x_n) / a_{n-2,n-2} \\
    &\vdots \\
    x_1 &= (b_1 - a_{12} x_2 - a_{13} x_3 - \cdots - a_{1,n} x_{n}) / a_{11}.
\end{align*}
$$

* $n^2$ flops. $\rightarrow$ level 2 operation in BLAS

* $\mathbf{A}$ can be accessed by row (`ij` loop) or column (`ji` loop).

## Implementation

* BLAS level 2 function: [`?BLAS.trsv`](http://www.netlib.org/lapack/explore-html/d6/d96/dtrsv_8f.html) (triangular solve with one right hand side: $\mathbf{A}x=b$).

* BLAS level 3 function: [`?BLAS.trsm`](http://www.netlib.org/lapack/explore-html/de/da7/dtrsm_8f.html) (matrix triangular solve, i.e., multiple right hand sides: $\mathbf{A}\mathbf{X}=\alpha\mathbf{C}\quad$ where $A$ is triangular, so $n$ times of $\mathbf{A}x=\alpha c_{j}$ is required).

* The BLAS triangular system solve is done *in place*, i.e., $\mathbf{b}$ is **overwritten** by $\mathbf{x}\;$. (Note that $\mathbf{b}$ and $\mathbf{x}$ is of same size)
```Julia
    # forward substitution
    for i=1:n
        for j=1:i-1
            b[i] -= A[i, j] * b[j]
        end
        b[i] /= A[i, i]
    end
    # backsolve
    for i=n:-1:1
        for j=i+1:n
            b[i] -= A[i, j] * b[j]
        end
        b[i] /= A[i, i]
    end
```

* Julia  
    - The left divide `\` operator in Julia is used for solving linear equations or least squares problem.  
    - If `A` is a triangular matrix, the command `A \ b` uses forward or backward substitution
        + Imagine $\frac{b}{A}=A^{-1}b$ to memorize.
    - Or we can call the BLAS wrapper functions directly: [`trsv!`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.BLAS.trsv!), [`trsv`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.BLAS.trsv), [`trsm!`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.BLAS.trsm!), [`trsm`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.BLAS.trsm)

In [2]:
using LinearAlgebra, Random

Random.seed!(123) # seed
n = 5
A = randn(n, n)
b = randn(n)
# a random matrix
A

5×5 Matrix{Float64}:
  1.19027   -0.664713   -0.339366   0.368002  -0.979539
  2.04818    0.980968   -0.843878  -0.281133   0.260402
  1.14265   -0.0754831  -0.888936  -0.734886  -0.468489
  0.459416   0.273815    0.327215  -0.71741   -0.880897
 -0.396679  -0.194229    0.592403  -0.77507    0.277726

In [3]:
Al = LowerTriangular(A) # does not create an extra matrix, as `Transpose(A)``

5×5 LowerTriangular{Float64, Matrix{Float64}}:
  1.19027     ⋅           ⋅          ⋅        ⋅ 
  2.04818    0.980968     ⋅          ⋅        ⋅ 
  1.14265   -0.0754831  -0.888936    ⋅        ⋅ 
  0.459416   0.273815    0.327215  -0.71741   ⋅ 
 -0.396679  -0.194229    0.592403  -0.77507  0.277726

In [4]:
fieldnames(typeof(Al))

(:data,)

In [5]:
Al.data

5×5 Matrix{Float64}:
  1.19027   -0.664713   -0.339366   0.368002  -0.979539
  2.04818    0.980968   -0.843878  -0.281133   0.260402
  1.14265   -0.0754831  -0.888936  -0.734886  -0.468489
  0.459416   0.273815    0.327215  -0.71741   -0.880897
 -0.396679  -0.194229    0.592403  -0.77507    0.277726

In [6]:
# same data
pointer(Al.data), pointer(A)

(Ptr{Float64} @0x0000000113b5ccb0, Ptr{Float64} @0x0000000113b5ccb0)

In [7]:
Al \ b # dispatched to BLAS function for triangular solve

5-element Vector{Float64}:
  1.28031359532452
 -4.485407033333146
  5.326125412123139
  0.446819508138921
 -3.091688163812573

In [8]:
# or use BLAS wrapper directly
BLAS.trsv('L', 'N', 'N', A, b)
# 'L' indicates that input matrix is lower triangular

5-element Vector{Float64}:
  1.28031359532452
 -4.485407033333146
  5.326125412123139
  0.446819508138921
 -3.091688163812573

In [9]:
?BLAS.trsv

```
trsv(ul, tA, dA, A, b)
```

Return the solution to `A*x = b` or one of the other two variants determined by [`tA`](@ref stdlib-blas-trans) and [`ul`](@ref stdlib-blas-uplo). [`dA`](@ref stdlib-blas-diag) determines if the diagonal values are read or are assumed to be all ones.


* Some other BLAS functions for triangular systems: [`trmv`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.BLAS.trmv), [`trmv!`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.BLAS.trmv!), [`trmm`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.BLAS.trmm), [`trmm!`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.BLAS.trmm!)

## Some algebraic facts of triangular system

* Eigenvalues of a triangular matrix $\mathbf{A}$ are diagonal entries $\lambda_i = a_{ii}$. 

* Determinant $\det(\mathbf{A}) = \prod_i a_{ii}$.

* The product of two upper (lower) triangular matrices is upper (lower) triangular.

* The inverse of an upper (lower) triangular matrix is upper (lower) triangular. ( if the inverse exists )

* The product of two unit upper (lower) triangular matrices is unit upper (lower) triangular.

* The inverse of a unit upper (lower) triangular matrix is unit upper (lower) triangular. ( if the inverse exists )

## Julia types for triangular matrices

[LowerTriangular](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.LowerTriangular), UnitLowerTriangular, 
[UpperTriangular](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.UpperTriangular), UnitUpperTriangular.  

In [12]:
using BenchmarkTools, LinearAlgebra, Random

Random.seed!(123) # seed
A = randn(1000, 1000);

In [13]:
# if we don't tell Julia it's triangular: O(n^3) complexity for finding eigenvalues of square matrix
# tril(A) returns a full triangular matrix, same as Matlab
@benchmark eigvals(tril($A))

# tril(A) is not same as Lowertriangular(A)
# tril(A) is a matrix type

BenchmarkTools.Trial: 83 samples with 1 evaluation.
 Range (min … max):  57.820 ms … 68.793 ms  ┊ GC (min … max): 0.00% … 4.86%
 Time  (median):     60.182 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   60.588 ms ±  2.248 ms  ┊ GC (mean ± σ):  1.27% ± 1.82%

   ▃ ▆ ▆  ▃▃▃ ▁▁▁ ▁ █▃▁    ▁                                   
  ▇█▄█▇█▄▇███▇███▇█▇███▇▄▄▁█▇▁▄▁▁▄▄▁▁▁▁▁▁▄▁▁▁▁▄▁▁▁▁▄▁▁▁▁▁▁▁▁▇ ▁
  57.8 ms         Histogram: frequency by time          68 ms <

 Memory estimate: 15.55 MiB, allocs estimate: 15.

In [21]:
B=tril(A)
typeof(B)


Matrix{Float64} (alias for Array{Float64, 2})

In [23]:
pointer(A)==pointer(B)

false

In [14]:
# if we tell Julia it's triangular: O(n) complexity
@benchmark eigvals(LowerTriangular($A))

# much faster and less memory allocation

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  1.997 μs … 793.644 μs  ┊ GC (min … max):  0.00% … 99.20%
 Time  (median):     2.501 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   4.115 μs ±  24.606 μs  ┊ GC (mean ± σ):  22.03% ±  3.71%

  █▆▄▃█▇▃▂                     ▃▅▅▄▃▂▂▂▂▁                     ▂
  █████████▇██▇▆▆▆▆▆▇▆▇▅▆▅▆▆▅▆██████████████▇██▇▆▇▆▆▆▆▅▆▅▅▅▆▆ █
  2 μs         Histogram: log(frequency) by time      8.16 μs <

 Memory estimate: 7.94 KiB, allocs estimate: 1.

In [15]:
@benchmark det(tril($A))

BenchmarkTools.Trial: 2088 samples with 1 evaluation.
 Range (min … max):  1.445 ms …   5.586 ms  ┊ GC (min … max):  0.00% … 46.38%
 Time  (median):     2.007 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   2.375 ms ± 807.040 μs  ┊ GC (mean ± σ):  16.54% ± 22.01%

          ▇█                                                   
  ▄▅▄▃▃▃▂▃███▆▄▄▄▄▄▃▃▄▄▃▃▃▃▂▃▂▂▂▂▂▂▂▂▁▂▁▂▁▁▃▅▄▃▃▂▃▂▂▂▂▂▂▂▃▂▂▂ ▃
  1.44 ms         Histogram: frequency by time        4.63 ms <

 Memory estimate: 7.64 MiB, allocs estimate: 3.

In [16]:
# if we tell Julia it's triangular: O(n) complexity
@benchmark det(LowerTriangular($A))

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.177 μs … 863.507 μs  ┊ GC (min … max):  0.00% … 99.49%
 Time  (median):     2.659 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.931 μs ±  25.164 μs  ┊ GC (mean ± σ):  24.87% ±  3.97%

  ▇▆▄▂█▆▄▂▁                   ▁▂▂▁                            ▂
  █████████▇█▇▆▆▇▅▅▇▆▅▆▆▆▆▅▅▅████████▇█▇▆▅▅▆▆▅▄▅▅▅▄▅▄▄▅▄▅▃▅▃▄ █
  2.18 μs      Histogram: log(frequency) by time      8.49 μs <

 Memory estimate: 7.94 KiB, allocs estimate: 1.

In [18]:
A = rand(5, 5)

5×5 Matrix{Float64}:
 0.117356    0.0866442  0.710067   0.372991    0.1636
 0.700676    0.983363   0.331878   0.00596027  0.43842
 0.413764    0.160028   0.495339   0.216016    0.891323
 0.280823    0.71423    0.0481896  0.944432    0.446089
 0.00489848  0.786108   0.505563   0.490008    0.364867

In [19]:
LowerTriangular(A)

5×5 LowerTriangular{Float64, Matrix{Float64}}:
 0.117356     ⋅         ⋅          ⋅         ⋅ 
 0.700676    0.983363   ⋅          ⋅         ⋅ 
 0.413764    0.160028  0.495339    ⋅         ⋅ 
 0.280823    0.71423   0.0481896  0.944432   ⋅ 
 0.00489848  0.786108  0.505563   0.490008  0.364867

In [20]:
LinearAlgebra.UnitLowerTriangular(A)

5×5 UnitLowerTriangular{Float64, Matrix{Float64}}:
 1.0          ⋅         ⋅          ⋅         ⋅ 
 0.700676    1.0        ⋅          ⋅         ⋅ 
 0.413764    0.160028  1.0         ⋅         ⋅ 
 0.280823    0.71423   0.0481896  1.0        ⋅ 
 0.00489848  0.786108  0.505563   0.490008  1.0

## Acknowledgment

Many parts of this lecture note is based on [Dr. Hua Zhou](http://hua-zhou.github.io)'s 2019 Spring Statistical Computing course notes available at <http://hua-zhou.github.io/teaching/biostatm280-2019spring/index.html>.